In [2]:
import torch
%load_ext autoreload
%autoreload 2

In [3]:
from model.threedepn import ThreeDEPNDecoder
from util.model import summarize_model

threedepn = ThreeDEPNDecoder()
print(summarize_model(threedepn))

   | Name         | Type             | Params  
-----------------------------------------------------
0  | bottleneck   | Sequential       | 197376  
1  | bottleneck.0 | Linear           | 65792   
2  | bottleneck.1 | ReLU             | 0       
3  | bottleneck.2 | Linear           | 131584  
4  | bottleneck.3 | ReLU             | 0       
5  | decoder1     | Sequential       | 8389376 
6  | decoder1.0   | ConvTranspose3d  | 8388864 
7  | decoder1.1   | BatchNorm3d      | 512     
8  | decoder1.2   | ReLU             | 0       
9  | decoder2     | Sequential       | 2097536 
10 | decoder2.0   | ConvTranspose3d  | 2097280 
11 | decoder2.1   | BatchNorm3d      | 256     
12 | decoder2.2   | ReLU             | 0       
13 | decoder3     | Sequential       | 524480  
14 | decoder3.0   | ConvTranspose3d  | 524352  
15 | decoder3.1   | BatchNorm3d      | 128     
16 | decoder3.2   | ReLU             | 0       
17 | decoder4     | Sequential       | 4097    
18 | decoder4.0   | ConvTranspose3

In [4]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train', filter_class='lamp')
val_dataset = ShapeNet('val', filter_class='lamp')
test_dataset = ShapeNet('test', filter_class='lamp')

print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
print(f'Length of val set: {len(val_dataset)}')  # expected output: 153540
print(f'Length of test set: {len(test_dataset)}')  # expected output: 64

Length of train set: 1854
Length of val set: 232
Length of test set: 232


In [5]:
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = test_dataset[231]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)
print(f'Target DF: {type(sample["target_df"])}')  # expected output: <class 'numpy.ndarray'>

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)
Target DF: <class 'numpy.ndarray'>


Output()

In [6]:
##################
#                #
#    TRAINING    #
#                #
##################

In [7]:
# TABLE VAD
from scripts import train
config = {
    'experiment_name': 'table_vad',
    'device': 'cuda:0',
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 1000,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'kl_weight_increase_every_epochs': 100,
    'kl_weight_increase_value': 0.01,
    'resume_ckpt': 'table_vad',
    'filter_class': 'table',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 4800
Training params: 3
[001/00024] train_loss: 0.174892 kl_loss: 0.490081 normal_loss: 0.160189
[002/00049] train_loss: 0.132340 kl_loss: 0.436066 normal_loss: 0.119258
[003/00074] train_loss: 0.125638 kl_loss: 0.381073 normal_loss: 0.114205
[005/00024] train_loss: 0.120445 kl_loss: 0.337990 normal_loss: 0.110305
[006/00049] train_loss: 0.114292 kl_loss: 0.321254 normal_loss: 0.104654
[007/00074] train_loss: 0.111616 kl_loss: 0.307387 normal_loss: 0.102394
[009/00024] train_loss: 0.108841 kl_loss: 0.294496 normal_loss: 0.100006
[010/00049] train_loss: 0.105368 kl_loss: 0.289730 normal_loss: 0.096676
[011/00074] train_loss: 0.100769 kl_loss: 0.285123 normal_loss: 0.092216
[013/00024] train_loss: 0.099397 kl_loss: 0.280643 normal_loss: 0.090978
[014/00049] train_loss: 0.098232 kl_loss: 0.276889 normal_loss: 0.089926
[015/00074] train_loss: 0.095730 kl_loss: 0.270200 normal_loss: 0.087624
[017/00024] train_loss: 0.093486 kl_loss: 0.264647 normal_loss: 0.0

In [21]:
# AIRPLANE VAD
from scripts import train
config = {
    'experiment_name': 'airplane_vad',
    'device': 'cuda:0',
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 1000,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'kl_weight_increase_every_epochs': 100,
    'kl_weight_increase_value': 0.0,
    'resume_ckpt': 'airplane_vad',
    'filter_class': 'airplane',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 3236
Training params: 3
[001/00048] train_loss: 0.123281 kl_loss: 0.483260 normal_loss: 0.108783
[003/00046] train_loss: 0.058216 kl_loss: 0.408448 normal_loss: 0.045963
[005/00044] train_loss: 0.048277 kl_loss: 0.339225 normal_loss: 0.038100
[007/00042] train_loss: 0.043332 kl_loss: 0.281702 normal_loss: 0.034881
[009/00040] train_loss: 0.040835 kl_loss: 0.237563 normal_loss: 0.033708
[011/00038] train_loss: 0.037442 kl_loss: 0.206272 normal_loss: 0.031254
[013/00036] train_loss: 0.037738 kl_loss: 0.180581 normal_loss: 0.032321
[015/00034] train_loss: 0.037525 kl_loss: 0.159066 normal_loss: 0.032753
[017/00032] train_loss: 0.035151 kl_loss: 0.138642 normal_loss: 0.030991
[019/00030] train_loss: 0.032855 kl_loss: 0.121132 normal_loss: 0.029221
[021/00028] train_loss: 0.031596 kl_loss: 0.106062 normal_loss: 0.028415
[023/00026] train_loss: 0.031367 kl_loss: 0.093590 normal_loss: 0.028559
[025/00024] train_loss: 0.030193 kl_loss: 0.084878 normal_loss: 0.0

In [6]:
#####################
#                   #
#    VISUALIZING    #
#                   #
#####################

In [7]:
from scripts.visualize import visualize_dataset_sample, visualize_ad, visualize_vad, visualize_vad_norm, visualize_vad_norm

In [142]:
import random
experiment = "airplane_vad"
experiment2 = "sofa_ad"
# experiment2 = "sofa_ad"
filter_class = "airplane"
index = 4123
index1 = random.choice(range(len(ShapeNet('train', filter_class = filter_class))))
index2 = random.choice(range(len(ShapeNet('train', filter_class = filter_class))))
a1 = 0.5
a2 = 1 - a1
#-------
# visualize_vad_norm(experiment2)
visualize_vad_norm(experiment)
visualize_ad(experiment, index1)
# visualize_vad_norm(experiment2)
# visualize_ad(experiment, index)
#-------
# visualize_vad_norm(experiment)
# visualize_vad_norm(experiment2)
# visualize_dataset_sample(filter_class, index)
#-------
# visualize_interpolation_ad(experiment, index1, index2, a1, a2)
# visualize_ad(experiment, index1)
# visualize_ad(experiment, index2)

Output()

Output()

In [8]:
####################
#                  #
#    EVALUATION    #
#                  #
####################

In [66]:
from scripts.evaluate import generate_samples, convert_df_to_point_cloud, chamfer_distance, MMD, convert_set_to_point_cloud
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import numpy as np

In [67]:
# MMD
MMD('airplane_vad', 'val', 'airplane', n_samples=10, device=torch.device('cuda:0'))

RuntimeError: CUDA out of memory. Tried to allocate 1.12 GiB (GPU 0; 8.00 GiB total capacity; 6.48 GiB already allocated; 0 bytes free; 6.76 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [39]:
# MMD

# generate n new samples
n = 10
samples = generate_samples('airplane_vad', n)
samples = samples.squeeze(1)


In [40]:
# get validation datasett
val = []
val_dataset = ShapeNet('val', filter_class='airplane')
for data_dict in val_dataset:
    target_df = torch.from_numpy(data_dict['target_df']).float()
    val.append(target_df)
val = torch.stack(val)

In [41]:
# visualize
input_mesh = marching_cubes(samples[8].detach().numpy(), level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Output()

In [42]:
# calculate chamfer distance between 1 generated sample and 1 sample from val
point_cloud = convert_df_to_point_cloud(samples[0])
point_cloud2 = convert_df_to_point_cloud(val[0])
chamfer_distance(point_cloud, point_cloud2)

tensor(0.8614)

In [14]:
# convert sets to pointclouds
samples_point_clouds = convert_set_to_point_cloud(samples)
val_point_clouds = convert_set_to_point_cloud(val)

In [18]:
# calculate mmd between the 2 samples
# Note: the final value changes a bit every time because every time a new point cloud is generated
# Note: mmd function takes a lot of time, needs optimization
mmd_value = mmd(samples_point_clouds, val_point_clouds)
mmd_value.mean()

1: 1.8070911169052124
2: 1.7960222959518433
3: 1.9385355710983276
4: 3.0144054889678955
5: 1.745727777481079
6: 1.736878752708435
7: 1.8000874519348145
8: 1.786746621131897
9: 2.1584560871124268
10: 1.9656767845153809


tensor(1.9750, device='cuda:0')